## Health Data analysis

Data analysis is crucial for businesses and organizations to make informed decisions from raw data. Here are the main types:

1\. Descriptive Analysis
------------------------

*   **What it does:** Summarizes historical data to show what has happened. Uses techniques like mean, median, mode, standard deviation, and range.

2\. Diagnostic Analysis
-----------------------

*   **What it does:** Explores why something happened by drilling down into data to identify patterns, correlations, and anomalies.

3\. Predictive Analysis
-----------------------

*   **What it does:** Uses historical data and statistical techniques to predict future outcomes.

4\. Prescriptive Analysis
-------------------------

*   **What it does:** Recommends actions to take by using optimization techniques to identify the best course of action.

5\. Exploratory Analysis
------------------------

*   **What it does:** Discovers patterns and relationships in data, often used in early stages to gain understanding and generate hypotheses.

6\. Inferential Analysis
------------------------

*   **What it does:** Uses statistical methods to draw conclusions about a population based on a sample of data.

7\. Causal Analysis
-------------------

*   **What it does:** Identifies cause-and-effect relationships between variables.

8\. Mechanistic Analysis
------------------------

*   **What it does:** Focuses on understanding the underlying mechanisms that drive a phenomenon.

### Setup Environment

In [ ]:
%run initializespark.ipynb

### Loading Data

In [ ]:
# Load the data into a dataframe
health_df = spark.read.parquet(
    "testdata/health_data_dev.parquet", header=True, inferSchema=True
)

# Show the first few rows of the dataframe
health_df.show()

### 1\. Descriptive Analysis
------------------------
Summarizes historical data to show what has happened. Uses techniques like mean, median, mode, standard deviation, and range.

In [ ]:
# Perform descriptive analysis
health_df.describe().show()

2\. Diagnostic Analysis
-----------------------

*   **What it does:** Explores why something happened by drilling down into data to identify patterns, correlations, and anomalies.

In [ ]:
from pyspark.sql.functions import col, corr

# Import necessary libraries

# Example: Find correlation between 'age' and 'BloodPressure'
age_bp_corr = health_df.select(corr("Age", "BloodPressure")).collect()[0][0]
print(f"Correlation between age and blood pressure: {age_bp_corr}")

# Identify anomalies using standard deviation
mean_bp = health_df.select("BloodPressure").groupBy().mean().collect()[0][0]
stddev_bp = (
    health_df.select("BloodPressure")
    .groupBy()
    .agg({"BloodPressure": "stddev"})
    .collect()[0][0]
)

# Define a threshold for anomalies (e.g., 3 standard deviations from the mean)
threshold = 3 * stddev_bp

# Filter out anomalies
anomalies = health_df.filter(
    (col("BloodPressure") > mean_bp + threshold)
    | (col("BloodPressure") < mean_bp - threshold)
)
print("Anomalies in blood pressure:")
anomalies.show()

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Select features and label
feature_columns = ["Age", "Height", "Weight"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(health_df)

# Split the data into training and test sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Initialize and train the linear regression model
lr = LinearRegression(featuresCol="features", labelCol="BloodPressure")
lr_model = lr.fit(train_data)

# Make predictions on the test set
predictions = lr_model.transform(test_data)

# Evaluate the model
evaluator = RegressionEvaluator(
    labelCol="BloodPressure", predictionCol="prediction", metricName="rmse"
)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data: {rmse}")

# Show some predictions
predictions.select("Age", "Height", "Weight", "BloodPressure", "prediction").show()

### 3\. Predictive Analysis
-----------------------

##### *   **What it does:** Uses historical data and statistical techniques to predict future outcomes.

In [ ]:
def train_and_evaluate_linear_regression(
    data, feature_columns, label_column, test_size=0.2, seed=42
):
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.regression import LinearRegression
    from pyspark.ml.evaluation import RegressionEvaluator

    # Assemble features
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    data = assembler.transform(data)

    # Split the data into training and test sets
    train_data, test_data = data.randomSplit([1 - test_size, test_size], seed=seed)

    # Initialize and train the linear regression model
    lr = LinearRegression(featuresCol="features", labelCol=label_column)
    lr_model = lr.fit(train_data)

    # Make predictions on the test set
    predictions = lr_model.transform(test_data)

    # Evaluate the model
    evaluator = RegressionEvaluator(
        labelCol=label_column, predictionCol="prediction", metricName="rmse"
    )
    rmse = evaluator.evaluate(predictions)
    print(f"Root Mean Squared Error (RMSE) on test data: {rmse}")

    # Show some predictions
    predictions.select(*feature_columns, label_column, "prediction").show()

    return lr_model, rmse, predictions


def predict_blood_pressure(lr_model, age, height, weight):
    from pyspark.ml.linalg import Vectors
    from pyspark.sql import Row

    # Create a single row dataframe with the input features
    input_data = spark.createDataFrame(
        [
            Row(
                Age=age,
                Height=height,
                Weight=weight,
                features=Vectors.dense([age, height, weight]),
            )
        ]
    )

    # Make prediction
    prediction = lr_model.transform(input_data)
    predicted_bp = prediction.select("prediction").collect()[0][0]
    return predicted_bp


# Example usage
lr_model, rmse, predictions = train_and_evaluate_linear_regression(
    health_df, ["Age", "Height", "Weight"], "BloodPressure"
)

In [ ]:
predicted_bp = predict_blood_pressure(lr_model, 44, 70, 168)
print(f"Predicted Blood Pressure: {predicted_bp}")

### 5\. Exploratory Analysis
------------------------

##### *   **What it does:** Discovers patterns and relationships in data, often used in early stages to gain understanding and generate hypotheses.

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Convert Spark DataFrame to Pandas DataFrame for easier plotting
health_pd_df = health_df.toPandas()

# Plotting pairplot to see relationships between variables
sns.pairplot(health_pd_df[["Age", "Height", "Weight", "BloodPressure"]])
plt.show()

# Plotting correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = health_pd_df[["Age", "Height", "Weight", "BloodPressure"]].corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

# Plotting distribution of BloodPressure
plt.figure(figsize=(10, 6))
sns.histplot(health_pd_df["BloodPressure"], kde=True)
plt.title("Distribution of Blood Pressure")
plt.xlabel("Blood Pressure")
plt.ylabel("Frequency")
plt.show()

# Boxplot to check for outliers in BloodPressure
plt.figure(figsize=(10, 6))
sns.boxplot(x=health_pd_df["BloodPressure"])
plt.title("Boxplot of Blood Pressure")
plt.xlabel("Blood Pressure")
plt.show()

### 6\. Inferential Analysis
------------------------

##### *   **What it does:** Uses statistical methods to draw conclusions about a population based on a sample of data.

In [ ]:
from scipy.stats import ttest_ind

# Convert Spark DataFrame to Pandas DataFrame for easier analysis
health_pd_df = health_df.select("Age", "Height", "Weight", "BloodPressure").toPandas()

# Separate the data into two groups based on Gender
older_bp = health_pd_df[health_pd_df["Age"] > 40]["BloodPressure"]
younger_bp = health_pd_df[health_pd_df["Age"] < 40]["BloodPressure"]

## Perform t-test to compare the means of BloodPressure between males and females
t_stat, p_value = ttest_ind(older_bp, younger_bp, nan_policy="omit")

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Interpret the results
alpha = 0.05
if p_value < alpha:
    print(
        "Reject the null hypothesis: There is a significant difference in Blood Pressure between older population and younger population."
    )
else:
    print(
        "Fail to reject the null hypothesis: There is no significant difference in Blood Pressure between older population and younger population."
    )

### Causal Analysis
-------------------

##### *   **What it does:** Identifies cause-and-effect relationships between variables.

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

# Convert Spark DataFrame to Pandas DataFrame for easier analysis
health_pd_df = health_df.select("Age", "Height", "Weight", "BloodPressure").toPandas()

# Perform Granger causality test to see if Age causes BloodPressure
max_lag = 4
test_result = grangercausalitytests(
    health_pd_df[["BloodPressure", "Age"]], max_lag, verbose=False
)

print(test_result)
# Perform Granger causality test to see if Weight causes BloodPressure
test_result = grangercausalitytests(
    health_pd_df[["BloodPressure", "Weight"]], max_lag, verbose=False
)

# Perform Granger causality test to see if Height causes BloodPressure
test_result = grangercausalitytests(
    health_pd_df[["BloodPressure", "Height"]], max_lag, verbose=False
)

### Mechanistic Analysis

In [ ]:
# Assemble features
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(health_df)

# Initialize and train the linear regression model
lr = LinearRegression(featuresCol="features", labelCol="BloodPressure")
lr_model = lr.fit(data)

# Print the coefficients and intercept
coefficients = lr_model.coefficients
intercept = lr_model.intercept
print(f"Coefficients: {coefficients}")
print(f"Intercept: {intercept}")

# Interpret the coefficients
for feature, coef in zip(feature_columns, coefficients):
    print(
        f"The coefficient for {feature} is {coef}, indicating the change in Blood Pressure for a unit change in {feature}."
    )